In [1]:

from Core.Tensor import Tensor
from Core.Operation import *
from Core.Model import *
from Core.Loss import *
from Core.Optimizer import *
import keras
class TwoLayerModel(Model):
    def __init__(self):
        super().__init__()
        self.linear1 = Linear(784, 128)
        self.linear2 = Linear(128, 10)
        self.relu = Relu()
        self.softmax = Softmax()
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x
    

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [3]:


model = TwoLayerModel()
loss_fn = MSE()
optimizer = SGD(model.get_parameters(), lr=0.001)

batch_size = 128
epochs = 20

for epoch in range(epochs):
    for i in range(0, len(x_train), batch_size):
        batch_x = x_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        batch_x = Tensor(batch_x)
        batch_y = Tensor(batch_y)
        
        output = model(batch_x)
        
        loss = loss_fn(output, batch_y)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
    output = model(x_test)
    acc = np.sum(np.argmax(output.data, axis=1) == np.argmax(y_test, axis=1)) / len(x_test)
    print("Epoch: %d, Loss: %.5f, Accuracy: %.5f" % (epoch+1, loss.data, acc))

Epoch: 1, Loss: 0.01574, Accuracy: 0.88780
Epoch: 2, Loss: 0.01141, Accuracy: 0.90560
Epoch: 3, Loss: 0.00983, Accuracy: 0.91430
Epoch: 4, Loss: 0.00863, Accuracy: 0.92070
Epoch: 5, Loss: 0.00766, Accuracy: 0.92530
Epoch: 6, Loss: 0.00683, Accuracy: 0.92900
Epoch: 7, Loss: 0.00624, Accuracy: 0.93260
Epoch: 8, Loss: 0.00582, Accuracy: 0.93650
Epoch: 9, Loss: 0.00548, Accuracy: 0.94010
Epoch: 10, Loss: 0.00522, Accuracy: 0.94380
Epoch: 11, Loss: 0.00499, Accuracy: 0.94600
Epoch: 12, Loss: 0.00477, Accuracy: 0.94820
Epoch: 13, Loss: 0.00460, Accuracy: 0.94940
Epoch: 14, Loss: 0.00445, Accuracy: 0.95090
Epoch: 15, Loss: 0.00432, Accuracy: 0.95310
Epoch: 16, Loss: 0.00417, Accuracy: 0.95430
Epoch: 17, Loss: 0.00404, Accuracy: 0.95540
Epoch: 18, Loss: 0.00394, Accuracy: 0.95710
Epoch: 19, Loss: 0.00383, Accuracy: 0.95780
Epoch: 20, Loss: 0.00373, Accuracy: 0.95860
